# Introduction

Welcome to the P1 Workspace with Group 14:
Will Mitchell
Trevor Gordon


Below are some prelimnary Helper methods that benifit the program as a whole with the starting code parameters

In [48]:
# Parameters
GROUP_ID = 'Group14'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Evil-P3.txt'

In [50]:
def parse_matrix(file_path : str):
  with open(file_path, mode="r") as file:
      new_matrix = []
      for line in file:
        new_line_array = line.replace("\n","").replace("\ufeff","").split(",")
        new_matrix.append(new_line_array)


  file.close()
  new_matrix[0][0] = str(new_matrix[0][0])[-1:]
  return new_matrix

parse_matrix(PUZZLE_PATH)



[['4', '?', '?', '?', '9', '?', '2', '1', '7'],
 ['?', '?', '?', '?', '?', '3', '8', '?', '?'],
 ['?', '?', '7', '?', '?', '?', '?', '?', '9'],
 ['?', '?', '9', '?', '?', '5', '?', '7', '6'],
 ['?', '?', '?', '?', '?', '?', '?', '?', '?'],
 ['5', '2', '?', '9', '?', '?', '3', '?', '?'],
 ['8', '?', '?', '?', '?', '?', '7', '?', '?'],
 ['?', '?', '4', '3', '?', '?', '?', '?', '?'],
 ['9', '3', '6', '?', '5', '?', '?', '?', '8']]

In [ ]:
def get_square_values(sudoku_state, cord_x, cord_y):
  #first box
  if cord_x <= 2 and cord_y <= 2:

  elif (2 < cord_x < 5) and (cord_y <= 2):






  print(answer)


In [ ]:
matrix = parse_matrix(PUZZLE_PATH)
if 6 in matrix[0]:
  return True
else:
  return False


In [10]:
class ga_node (matrix):
    def __init__(self, cord_x, cord_y):
        self.cord_x = cord_x

SyntaxError: invalid syntax (451626592.py, line 1)

# Helper Functions


In [73]:
def get_row_values(sudoku_state, row, col):
  value = sudoku_state[row][col]
  my_set = set(sudoku_state[row])
  return my_set

print(get_row_values(parse_matrix(PUZZLE_PATH),0,0))

{'9', '1', '?', '7', '4', '2'}


In [82]:
def get_col_values(sudoku_state, row, col):
  value = sudoku_state[row][col]
  my_set = set()
  for i in range(9):
    my_set.add(sudoku_state[i][col])

  return my_set

print(get_col_values(parse_matrix(PUZZLE_PATH),0,1))

{'?', '2', '3'}


In [79]:
def create_ga_state(raw_sudoku_state):


_IncompleteInputError: incomplete input (2535199512.py, line 1)